In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt

In [ ]:
df = pd.read_excel("../Validation.xlsx")

df.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)
df.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)
df_valid = pd.concat([
    df[['Emotion', 'sentence']],
    df[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)

df_valid = df_valid.dropna()

In [ ]:
def get_sent_embedding(model, embedding_size, tokenized_words):
    feature_vec = np.zeros((embedding_size,), dtype="float32")
    n_words = 0
    for word in tokenized_words:
        if word in model.wv.key_to_index:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
df_train = pd.read_csv('../selected_data.csv')

In [ ]:
from joblib import load
svm_model = load("../../svm_model.joblib")

In [ ]:
df_valid = pd.read_csv("../validation_data.csv")
okt = Okt()

temp = []
for i in df_valid['sentence']:
    temp.append(okt.morphs(i))

df_valid['token'] = temp

In [ ]:
import numpy as np
model = load("../../model/word_model.joblib")
# 검증 데이터의 임베딩 생성
X_valid = [get_sent_embedding(model, 1000, tokens) for tokens in df_valid['token']]

# 실제 레이블
y_valid = df_valid['Emotion']
# SVM 모델로 예측 수행
y_pred = svm_model.predict(X_valid)
from sklearn.metrics import classification_report, confusion_matrix

# 성능 보고서
print("Classification Report:")
print(classification_report(y_valid, y_pred))

# 혼동 행렬
print("Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

emotion_mapping_inverse = {0: "기쁨", 1: "당황", 2: "불안", 3: "슬픔", 4: "상처", 5: "분노"}

# 혼동 행렬 생성
conf_matrix = confusion_matrix(y_valid, y_pred)

# 시각화
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=emotion_mapping_inverse.values(), 
            yticklabels=emotion_mapping_inverse.values())
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
